In [1]:
from konlpy.tag import Mecab
import pandas as pd
from itertools import islice
import re

In [6]:
df = pd.read_csv("../data/video_sample_test.csv", encoding="utf-8-sig", index_col=0)
df.head()

,video_id,text,true,predict
0,q5g1V6B7aiE,불티나게 팔린 어묵탕 추천순위 TOP10 #대림선 #고래사어묵 #cj제일제당 #범...,방송 광고/이벤트,NaN
1,BY6uCMhkZFo,Cyber Space 1-5 Dropaholic ( Alpha Version ) S...,해외영상,NaN
2,yoeTDrUB-_o,[ 5 WORDS COMPILATION Learn 5 Korean Words Phr...,해외영상,NaN
3,2CpgC1K7jNI,대포죽순이요 푸바오 핸펀몰카사건 푸바오 이야기 FUBAO PANDA CUTE ANI...,라이프스타일 일상/vlog,NaN
4,KiRMy2Bu5SU,嫁に海老を食べさせられた甲殻類アレルギーの俺 → 発作を起こした俺がその場で倒れて意識不明に...,해외영상,NaN


In [2]:
mecab = Mecab(dicpath='C:\Program Files (x86)\MeCab\dic\ipadic')


Exception: The MeCab dictionary does not exist at "C:\Program Files (x86)\MeCab\dic\ipadic". Is the dictionary correctly installed?
You can also try entering the dictionary path when initializing the Mecab class: "Mecab('/some/dic/path')"

In [ ]:

class Predicate():
    def __init__(self, stopwords_path="../usedata/stopwords_for_predicate.txt",
                 mecab_dic_path=None, match_poss=None, not_match_poss=None):

        # Mecab 형태소 분석기 초기화
        if mecab_dic_path:
            self.mecab = Mecab(dicpath=mecab_dic_path)
        else:
            self.mecab = Mecab()

        ### 불용어
        self.stopwords = set()
        with open(stopwords_path, "rt", encoding="utf-8-sig") as f:
            while True:
                text = f.readline()
                self.stopwords.add(text.strip())
                if not text:
                    break
        self.stopwords = list(self.stopwords)
        
        if not match_poss:
            self.match_poss = [["XR", "XSA+EP"], ["VV+EP", "EF"], ["VV+EP", "ETM"],
                            ["NNG", "XSV+EF"], ["MAG", "XSA+ETM"], ["NNG", "XSA+EC"], 
                            ["VV+EC", "VX+EP"], ["VV+EC", "VX+EF"], ["VV+ETM", "NNB", "VA"],
                            ["NNG", "XSV+ETM"], ["NNG", "XSA+EP"], ["VA", "EP"],
                            ["NNG", "XSV+EP"],  ["MAG", "VV+EC"], ["VV+ETM", "NNB", "VV"], 
                            ["VV", "EP+EP"], ["NNG", "XSA+ETM"], ["VV+EC", "VX"],
                            ["JKS", "VV+EC", "VX"], ["VV+EC", "VV+EC", "VX"], ["VV+EP", "EC"], 
                            ['NNG', 'XSV+EC'], ['NNG', 'NNG', 'XSA+ETM']]
        elif isinstance(match_poss, list):
            raise ValueError("Invalid type of match_poss. List format only supported")

        if not not_match_poss:
            self.not_match_poss = [["NNG", "XSA+ETM", "XPN"], ["NNG", "XSV+ETM", "NNG"], 
                                ["NNG", "XSV", "SY"], ["NNG", "VV+EC", "NNG"], ["JKS", "NNG", "XSV+ETM"],
                                ["NNG", "VCP+EP", "EF"], ["NNG", "XSV+ETM", "XPN"],
                                ['VCP+ETM', 'NNG', 'XSV+EC'], ['NNG', 'XSA+ETM', 'NNB']]
        elif isinstance(not_match_poss, list):
            raise ValueError("Invalid type of not_match_poss. List format only supported")


    
    def predict(self, keyword, related, days=31, size=20000, top=10):
        
        datas = self.get_data(keyword, related, days=days, size=size)
        if datas is None:
            return dict()
        verbs = dict()
        for data in datas:
            results = self.convert_to_da_form(data)
            for result in results:
                if result[0] not in verbs:
                    verbs[result[0]] = 1
                else:
                    verbs[result[0]] += 1
        # 1보다 큰 항목만 필터링하고, 내림차순으로 정렬
        filtered_verbs = {k: v for k, v in islice(sorted(verbs.items(), key=lambda item: item[1], reverse=True), top) if v > 1}
        return filtered_verbs

    def filter_by_common_pattern(self, data):
        result = list()
        for i, (word1, tag1) in enumerate(data):
            pass_tag = False
            for j, (word2, tag2) in enumerate(data):
                if i == j:  # 자기 자신은 제외
                    continue
                if (set(tag1) and set(tag2)) and (len(tag1) < len(tag2)):
                    pass_tag = True
                    break
            if not pass_tag:
                result.append((word1, tag1))
        return result
    
    def clean_text(self, text):
        # 해시태그 패턴
        hashtag_pattern = r'#\w+'
        
        # URL 패턴 (http, https, www로 시작하는 URL)
        url_pattern = r'https?://\S+|www\.\S+'
        
        # 해시태그와 URL 제거
        text_without_hashtags = re.sub(hashtag_pattern, ' ', text)
        clean_text = re.sub(url_pattern, ' ', text_without_hashtags)
        
        # | 특수문자 제거
        clean_text = clean_text.replace("|", " ").strip()
        clean_text = clean_text.replace("ㅣ", " ").strip()
        # 양쪽 공백 제거
        clean_text = clean_text.replace(" ", "[MASK]").strip()
        return clean_text
    
    def convert_to_da_form(self, text):
        
        # 입력값이 None인지 확인
        if text is None:
            return list()  # 빈 리스트나 기본 값을 반환
        
        text = self.clean_text(text)

        # 품사 태깅
        pos = self.mecab.pos(text)
        
        # 형태소 태그들만 추출
        # tags = [tag for token, tag in pos if token not in self.stopwords]
        tags = [tag for _, tag in pos]

        # 필터링 결과
        filtered_pos = []
        i = 0
        while i < len(pos):
            matched = False
            for pattern in self.not_match_poss:
                pattern_length = len(pattern)
                if tags[i:i + pattern_length] == pattern:
                    i += pattern_length  # 패턴 길이만큼 건너뜀
                    filtered_pos.append(("삭제됨", "DEL"))  # 삭제된 부분에 '삭제됨' 추가
                    matched = True
                    break
            if not matched:
                filtered_pos.append(pos[i])
                i += 1
        verbs = list()
        
        for match_pos in self.match_poss:
            # 슬라이딩 윈도우 방식으로 연속으로 존재하는 VV, EC, VX 찾기
            for i in range(len(filtered_pos) - len(match_pos) + 1):
                window = filtered_pos[i:i + len(match_pos)]
                window_tags = [tag for _, tag in window]
                
                if window_tags == match_pos:
                    text = "".join([i[0] for i in window])
                    if text in self.stopwords:
                        continue
                    
                    last_pos = window[-1][-1].split("+")[-1]
                    if last_pos in ["EP", "VV", "VX", "VA", "XSV", "JX", "ETM"]:
                        if text.endswith(tuple(["을", "는"])):
                            text = text[:-1]
                        text += "다"
                    if text.endswith(tuple(["합니다", "하겠다", "하였다", "했었다", "할까요", "해야다", 
                                            "한다면", "해서다", "해봤다", "합시다", "해라다", "하잖아", 
                                            "해도다", "한지다", "할까다", "합니까", "할런다", "합네다", 
                                            "할리다", "할려다", "할런지", "할려나", "할려고"])):
                        text = text[:-3] + "하다"
                    elif text.endswith(tuple(["하고있다", "해버렸다", "해드려요", "한다면다", "할까요다"])):
                        text = text[:-4] + "하다"
                    elif text.endswith(tuple(["올라갈까"])):
                        text = text[:-4] + "오르다"
                    elif text.endswith(tuple(["겨났는지다"])):
                        text = text[:-5] + "기다"
                    elif text.endswith(tuple(["져버렸다"])):
                        text = text[:-4] + "졌다"
                    elif text.endswith(tuple(["켰으니까"])):
                        text = text[:-4] + "키다"
                    elif text.endswith(tuple(["한다", "한가", "했다", "해요", "해서", "할까", "해라", 
                                              "해다", "한데", "해줘", "할다", "항다", "햇다", "할지",
                                              "할려"])):
                        text = text[:-2] + "하다"
                    elif text.endswith(tuple(["려다", "려줘"])):
                        text = text[:-2] + "리다"
                    elif text.endswith(tuple(["와다"])):
                        text = text[:-2] + "왔다"
                    elif text.endswith(tuple(["싶어"])):
                        text = text[:-2] + "싶다"
                    elif text.endswith(tuple(["깼다"])):
                        text = text[:-2] + "깨다"
                    elif text.endswith(tuple(["있어"])):
                        text = text[:-2] + "있다"
                    elif text.endswith(tuple(["춰다", "췄다"])):
                        text = text[:-2] + "추다"
                    elif text.endswith(tuple(["웠다", "워다"])):
                        text = text[:-2] + "우다"
                    elif text.endswith(tuple(["녀다"])):
                        text = text[:-2] + "녔다"
                    elif text.endswith(tuple(["갔다"])):
                        text = text[:-2] + "가다"
                    elif text.endswith(tuple(["켜다"])):
                        text = text[:-2] + "켰다"
                    elif text.endswith(tuple(["셨", "셧"])):
                        text = text[:-1] + "시다"
                    elif text.endswith(tuple(["셨다"])):
                        text = text[:-2] + "시다"
                    elif text.endswith(tuple(["셨습니까"])):
                        text = text[:-4] + "시다"
                    elif text.endswith(tuple(["져다"])):
                        text = text[:-2] + "졌다"
                    elif text.endswith(tuple(["탔다"])):
                        text = text[:-2] + "타다"
                    elif text.endswith(tuple(["겨다"])):
                        text = text[:-2] + "기다"
                    elif text.endswith(tuple(["겨났는다"])):
                        text = text[:-4] + "기다"
                    elif text.endswith(tuple(["킨다", "킬까"])):
                        text = text[:-2] + "키다"
                    elif text.endswith(tuple(["섰다"])):
                        text = text[:-2] + "서다"
                    elif text.endswith(tuple(["줬다", "준다", "줄게"])):
                        text = text[:-2] + "주다"
                    elif text.endswith(tuple(["냈다"])):
                        text = text[:-2] + "내다"
                    elif text.endswith(tuple(["꼈다"])):
                        text = text[:-2] + "끼다"
                    elif text.endswith(tuple(["꼈다다", "끼시다"])):
                        text = text[:-3] + "끼다"
                    elif text.endswith(tuple(["음"])):
                        text = text[:-1] + "으다"
                    elif text.endswith(tuple(["혀다", "혔다"])):
                        text = text[:-2] + "히다"
                    elif text.endswith(tuple(["졌다", "진다"])):
                        text = text[:-2] + "지다"
                    elif text.endswith(tuple(["려"])):
                        text = text[:-1] + "리다"
                    elif text.endswith(tuple(["드다"])):
                        text = text[:-2] + "들다"
                    elif text.endswith(tuple(["써다", "썼다"])):
                        text = text[:-2] + "쓰다"
                    elif text.endswith(tuple(["러워", "런다"])):
                        text = text[:-2] + "럽다"
                    elif text.endswith(tuple(["됨"])):
                        text = text[:-1] + "되다"
                    elif text.endswith(tuple(["일다", "여다", "려다", "였다", "였어"])):
                        text = text[:-2] + "이다"
                    elif text.endswith(tuple(["떠다"])):
                        text = text[:-2] + "뜨다"
                    elif text.endswith(tuple(["띈다"])):
                        text = text[:-2] + "띄다"
                    elif text.endswith(tuple(["둬라"])):
                        text = text[:-2] + "두다"
                    elif text.endswith(tuple(["되고있다", "되어있다"])):
                        text = text[:-4] + "되다"
                    elif text.endswith(tuple(["나버렸다"])):
                        text = text[:-4] + "났다"
                    elif text.endswith(tuple(["뎌야한다"])):
                        text = text[:-4] + "디다"
                    elif text.endswith(tuple(["려드리다", "릴게요다"])):
                        text = text[:-4] + "리다"
                    elif text.endswith(tuple(["들으며다"])):
                        text = text[:-4] + "듣다"
                    elif text.endswith(tuple(["됩니다", "된다면", "될지다", "된다던"])):
                        text = text[:-3] + "되다"
                    elif text.endswith(tuple(["이었다", "여줄까", "여봐요"])):
                        text = text[:-3] + "이다"
                    elif text.endswith(tuple(["폈는데다", "펴봤더니"])):
                        text = text[:-4] + "피다"
                    elif text.endswith(tuple(["려드릴께요", "려드릴게요"])):
                        text = text[:-5] + "리다"
                    elif text.endswith(tuple(["여드릴께요", "여드릴게요"])):
                        text = text[:-5] + "이다"
                    elif text.endswith(tuple(["라갈까"])):
                        text = text[:-3] + "르다"
                    elif text.endswith(tuple(["봅니다", "볼게요", "볼까요"])):
                        text = text[:-3] + "보다"
                    elif text.endswith(tuple(["타봤다"])):
                        text = text[:-3] + "타다"
                    elif text.endswith(tuple(["로운다"])):
                        text = text[:-3] + "롭다"
                    elif text.endswith(tuple(["진다면"])):
                        text = text[:-3] + "지다"
                    elif text.endswith(tuple(["개놓다"])):
                        text = text[:-3] + "개다"
                    elif text.endswith(tuple(["사볼까"])):
                        text = text[:-3] + "사다"
                    elif text.endswith(tuple(["다운다"])):
                        text = text[:-3] + "답다"
                    elif text.endswith(tuple(["췄다"])):
                        text = text[:-2] + "추다"
                    elif text.endswith(tuple(["춰줄게", "춰줄께"])):
                        text = text[:-3] + "추다"
                    elif text.endswith(tuple(["립니다", "릴까다", "려서다", "려볼까"])):
                        text = text[:-3] + "리다"
                    elif text.endswith(tuple(["겨봤다", "겼으니"])):
                        text = text[:-3] + "기다"
                    elif text.endswith(tuple(["져지다"])):
                        text = text[:-3] + "졌다"
                    elif text.endswith(tuple(["워봤다", "웠더니"])):
                        text = text[:-3] + "우다"
                    elif text.endswith(tuple(["나봤다"])):
                        text = text[:-3] + "났다"
                    elif text.endswith(tuple(["뎌야다"])):
                        text = text[:-3] + "디다"
                    elif text.endswith(tuple(["웁시다"])):
                        text = text[:-3] + "울다"
                    elif text.endswith(tuple(["않아요"])):
                        text = text[:-3] + "않다"
                    elif text.endswith(tuple(["켜야다"])):
                        text = text[:-3] + "키다"
                    elif text.endswith(tuple(["났다"])):
                        text = text[:-2] + "나다"
                    elif text.endswith(tuple(["켰다", "킨다", "킬다"])):
                        text = text[:-2] + "키다"
                    elif text.endswith(tuple(["샀다"])):
                        text = text[:-2] + "사다"
                    elif text.endswith(tuple(["켜"])):
                        text = text[:-1] + "키다"
                    elif text.endswith(tuple(["해"])):
                        text = text[:-1] + "하다"
                    elif text.endswith(tuple(["봐"])):
                        text = text[:-1] + "보다"
                    elif text.endswith(tuple(["기", "냐", "네", "죠", "어", "지", "나", "고", "니"])):
                        text = text[:-1] + "다"
                    elif text.endswith(tuple(["으니까다", "잖아요다"])):
                        text = text[:-4] + "다"
                    elif text.endswith(tuple(["러운다", "러우다"])):
                        text = text[:-3] + "럽다"
                    elif text.endswith(tuple(["네요", "셨다", "슴다", "어요", "다구", "라다", "렸나", "을까", \
                        "다면", "는데", "잖아", "다고", "었다", "어다", "쥬다", "나다", "고다", "았다", "겠다", \
                        "은다" "다다", "던다", "나요", "줘요", "다가", "지만", "더니", "였니", "는다", "습니", \
                        "으면", "습다", "시다", "으다"])):
                        text = text[:-2] + "다"
                    elif text.endswith(tuple(["습니다", "다고다", "읍니다", "더니다", "는데요", \
                        "냐구요", "아겠네", "지만다", "는데다", "구나다", "다가다", "리면다", "을까다", \
                        "냐고요", "으시다"])):
                        text = text[:-3] + "다"
                    elif text.endswith(tuple(["됐다", "된다", "될다", "될까", "돼다"])):
                        text = text[:-2] + "되다"
                    elif text.endswith(tuple(["돼"])):
                        text = text[:-2] + "되다"
                        
                    ### 2번째 처리(과거시제)
                    if text.endswith(tuple(["합니다", "한데다", "할게요"])):
                        text = text[:-3] + "하다"
                    elif text.endswith(tuple(["줬다"])):
                        text = text[:-2] + "주다"
                    elif text.endswith(tuple(["다다", "는다", "더다"])):
                        text = text[:-2] + "다"
                    elif text.endswith(tuple(["려드리다", "려야하다", "렸잖아요"])):
                        text = text[:-4] + "리다"
                    elif text.endswith(tuple(["춰다", "췄다"])):
                        text = text[:-2] + "추다"
                    elif text.endswith(tuple(["립니다", "렸더니", "릴게요"])):
                        text = text[:-3] + "리다"
                    elif text.startswith(tuple(["지으"])):
                        text = "짓" + text[2:]
                    elif text.endswith(tuple(["섰다"])):
                        text = text[:-2] + "서다"
                    elif text.startswith(tuple(["아무리"])):
                        text = text[3:]
                    elif text.endswith(tuple(["렸다", "랐다", "린다"])):
                        text = text[:-2] + "리다"
                    elif text.endswith(tuple(["였다"])):
                        text = text[:-2] + "이다"
                    elif text.endswith(tuple(["웠다"])):
                        text = text[:-2] + "우다"
                    elif text.endswith(tuple(["떴다"])):
                        text = text[:-2] + "뜨다"
                    elif text.endswith(tuple(["갔다"])):
                        text = text[:-2] + "가다"
                    elif text.endswith(tuple(["겼다"])):
                        text = text[:-2] + "기다"
                    elif text.endswith(tuple(["켰다", "켜야"])):
                        text = text[:-2] + "키다"
                    elif text.endswith(tuple(["쳤다", "친다"])):
                        text = text[:-2] + "치다"
                    elif text.endswith(tuple(["혀다", "혔다"])):
                        text = text[:-2] + "히다"
                    elif text.endswith(tuple(["쳤지만"])):
                        text = text[:-3] + "치다"
                    elif text.endswith(tuple(["끼시다"])):
                        text = text[:-3] + "끼다"
                    elif text.endswith(tuple(["났다"])):
                        text = text[:-2] + "나다"
                    elif text.endswith(tuple(["탔다"])):
                        text = text[:-2] + "타다"
                    elif text.endswith(tuple(["봤다", "봐다"])):
                        text = text[:-2] + "보다"
                    elif text.endswith(tuple(["졌다", "졌나"])):
                        text = text[:-2] + "지다"
                    elif text.endswith(tuple(["냈다"])):
                        text = text[:-2] + "내다"
                    elif text.endswith(tuple(["왔다", "왔나", "와라"])):
                        text = text[:-2] + "오다"
                    elif text.endswith(tuple(["꿨다"])):
                        text = text[:-2] + "꾸다"
                    elif text.endswith(tuple(["뤘다"])):
                        text = text[:-2] + "루다"
                    elif text.endswith(tuple(["해"])):
                        text = text[:-1] + "하다"
                    elif text.endswith(tuple(["갔었다", "갔다다", "갑니다"])):
                        text = text[:-3] + "가다"
                    elif text.endswith(tuple(["났었다", "났구나", "났지만"])):
                        text = text[:-3] + "나다"
                    elif text.endswith(tuple(["오시다"])):
                        text = text[:-3] + "오다"
                    elif text.endswith(tuple(["올랐다"])):
                        text = text[:-3] + "오르다"
                    elif text.endswith(tuple(["몰랐다"])):
                        text = text[:-3] + "모르다"
                    elif text.endswith(tuple(["늘렀다"])):
                        text = text[:-3] + "누르다"
                    elif text.endswith(tuple(["질렀다"])):
                        text = text[:-3] + "지르다"
                    elif text.endswith(tuple(["질렀길래"])):
                        text = text[:-4] + "지르다"
                    elif text.endswith(tuple(["질러야지다"])):
                        text = text[:-5] + "지르다"
                    elif text.endswith(tuple(["불렀다"])):
                        text = text[:-3] + "부르다"
                    elif text.endswith(tuple(["샀다"])):
                        text = text[:-2] + "사다"
                    elif text.endswith(tuple(["찼다"])):
                        text = text[:-2] + "차다"
                    elif text.endswith(tuple(["뗐다"])):
                        text = text[:-2] + "떼다"
                    elif text.endswith(tuple(["됐다", "된다", "될다", "될까", "돼다", "돼야"])):
                        text = text[:-2] + "되다"
                    elif text.endswith(tuple(["한다", "했다", "해도", "해야", "할다"])):
                        text = text[:-2] + "하다"

                    if text not in ["하다", "보다", "받다", "되다", "봤다", "나다", "났다", \
                        "가다", "있다", "없다", "많다", "그렇다", "그랬다", "저랬다"]:
                        if text.strip().endswith("다"):
                            verbs.append((text, match_pos))
        verbs = self.filter_by_common_pattern(verbs)

        return verbs